[Reference](https://rajansahu713.medium.com/building-a-fastapi-application-with-postgresql-and-docker-5cf67e383554)

# Setting Up the Database

In [1]:
from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "postgresql+asyncpg://postgres:123456@db:5432/demo"
engine = create_async_engine(DATABASE_URL, echo=True)
SessionLocal = sessionmaker(bind=engine, class_=AsyncSession, expire_on_commit=False)
Base = declarative_base()

# Creating the Dockerfile
```
FROM python:3.11
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

# Docker Compose Configuration
```
version: '3.11'

services:
  db:
    image: postgres
    environment:
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: 123456
      POSTGRES_DB: demo
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data
  web:
    build: ./app
    command: uvicorn main:app --host 0.0.0.0 --port 8000 --reload
    volumes:
      - ./app:/app
    ports:
      - "8000:8000"
    depends_on:
      - db
volumes:
  postgres_data:
```

# Defining Models and Schemas

In [3]:
import sqlalchemy
from databases import Base

class Item(Base):
    __tablename__ = "items
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True, index=True)
    name = sqlalchemy.Column(sqlalchemy.String, index=True)
    description = sqlalchemy.Column(sqlalchemy.String, index=True)

In [4]:
from pydantic import BaseModel

class ItemBase(BaseModel):
    name: str
    description: str
class ItemCreate(ItemBase):
    pass
class ItemResponse(ItemBase):
    id: int
    class Config:
        from_attributes = True

In [5]:
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy.future import select
from databases import Base, SessionLocal, engine
from schemas import ItemResponse, ItemCreate
from models import Item
from contextlib import asynccontextmanager


# Function to create the database tables
async def create_all():
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)
@asynccontextmanager
async def lifespan(app: FastAPI):
    await create_all()
    yield
app = FastAPI(lifespan=lifespan)
# Dependency
async def get_db():
    async with SessionLocal() as session:
        yield session
@app.post("/items/", response_model=ItemResponse)
async def create_item(item: ItemCreate, db: AsyncSession = Depends(get_db)):
    db_item = Item(name=item.name, description=item.description)
    db.add(db_item)
    await db.commit()
    await db.refresh(db_item)
    return db_item
@app.get("/items/{item_id}", response_model=ItemResponse)
async def read_item(item_id: int, db: AsyncSession = Depends(get_db)):
    result = await db.execute(select(Item).where(Item.id == item_id))
    item = result.scalars().first()
    if item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    return item
@app.put("/items/{item_id}", response_model=ItemResponse)
async def update_item(item_id: int, item: ItemCreate, db: AsyncSession = Depends(get_db)):
    result = await db.execute(select(Item).where(Item.id == item_id))
    db_item = result.scalars().first()
    if db_item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    db_item.name = item.name
    db_item.description = item.description
    await db.commit()
    await db.refresh(db_item)
    return db_item
@app.delete("/items/{item_id}", response_model=ItemResponse)
async def delete_item(item_id: int, db: AsyncSession = Depends(get_db)):
    result = await db.execute(select(Item).where(Item.id == item_id))
    db_item = result.scalars().first()
    if db_item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    await db.delete(db_item)
    await db.commit()
    return db_item

```
docker-compose up --build
```